# Recommender Systems HW 1 

Marla Goodman  9/7/2022

Assignment: choose at least six recent popular items (movie, books, television programs, courses, or other items that you find interesting).  Design a survey asking respondents to rate each item that they have seen on a scale of 1 to 5.  Send your survey out to family, friends, and/or class mates.  You should get responses from at least five people that you know.  There needs to be at least one respondent with at least two unrated items.  Your task is to use the global baseline estimate algorithm to determine ratings for unseen items, then recommend an item to at least one of your survey respondents.

#### Step 1. Get and organize data

In [94]:
#import pandas
import pandas as pd

#read in CSV
df=pd.read_csv('https://raw.githubusercontent.com/MarlaGoodman/DAVRecommenderSystems/main/SitComs.csv')

#show df
df

,Unnamed: 0,Parks&Rec,ModernFamily,TheOffice,Brooklyn99,BigBangTheory,30Rock
0,Phil,3.0,3.0,NaN,1.0,5.0,NaN
1,Ruth,1.0,NaN,NaN,1.0,NaN,NaN
2,Jill,2.0,4.0,2.0,NaN,2.0,1.0
3,Amy,5.0,3.0,4.0,5.0,5.0,3.0
4,Ian,NaN,1.0,4.0,1.0,NaN,4.0


In [95]:
#rename respondent column to 'Respondent'
df.rename(columns = {'Unnamed: 0':'Respondent'}, inplace = True)

#show df
df

,Respondent,Parks&Rec,ModernFamily,TheOffice,Brooklyn99,BigBangTheory,30Rock
0,Phil,3.0,3.0,NaN,1.0,5.0,NaN
1,Ruth,1.0,NaN,NaN,1.0,NaN,NaN
2,Jill,2.0,4.0,2.0,NaN,2.0,1.0
3,Amy,5.0,3.0,4.0,5.0,5.0,3.0
4,Ian,NaN,1.0,4.0,1.0,NaN,4.0


In [96]:
#make Respondent column the index
df2=df.set_index('Respondent')

#show df2
df2

,Parks&Rec,ModernFamily,TheOffice,Brooklyn99,BigBangTheory,30Rock
Respondent,,,,,,
Phil,3.0,3.0,NaN,1.0,5.0,NaN
Ruth,1.0,NaN,NaN,1.0,NaN,NaN
Jill,2.0,4.0,2.0,NaN,2.0,1.0
Amy,5.0,3.0,4.0,5.0,5.0,3.0
Ian,NaN,1.0,4.0,1.0,NaN,4.0


#### Step 2. Get average for TV shows (sum of all rankings/number of cells with numbers)

In [97]:
#get sum of df2
sum_df2=df2.sum().sum()

#get number of cells with numbers in them
cell_number=df2.count().sum()

#get tv average
tv_avg=(sum_df2/cell_number)

tv_avg

2.857142857142857

#### Step 3. Get mean of every row and subtract all the means by the TV average

In [98]:
row_mean=df2.mean(axis=1)
row_mean

Respondent
Phil    3.000000
Ruth    1.000000
Jill    2.200000
Amy     4.166667
Ian     2.500000
dtype: float64

In [99]:
user_deviation=(row_mean-tv_avg)

user_deviation

Respondent
Phil    0.142857
Ruth   -1.857143
Jill   -0.657143
Amy     1.309524
Ian    -0.357143
dtype: float64

#### Step 4. Get mean of TV show columns and subtract all those means by the TV average

In [100]:
column_mean=df2.mean()

column_mean

Parks&Rec        2.750000
ModernFamily     2.750000
TheOffice        3.333333
Brooklyn99       2.000000
BigBangTheory    4.000000
30Rock           2.666667
dtype: float64

In [101]:
tv_deviation = (column_mean-tv_avg)

tv_deviation

Parks&Rec       -0.107143
ModernFamily    -0.107143
TheOffice        0.476190
Brooklyn99      -0.857143
BigBangTheory    1.142857
30Rock          -0.190476
dtype: float64

#### Now we have the whole TV average, the user deviations, and the TV show deviations

#### Step 5. Get Global Baseline Estimate

Global Baseline Estimate = TV average + user deviation + tv deviation

In [102]:
#see all compenents to the equation

print(tv_avg)
print(user_deviation)
print(tv_deviation)

2.857142857142857
Respondent
Phil    0.142857
Ruth   -1.857143
Jill   -0.657143
Amy     1.309524
Ian    -0.357143
dtype: float64
Parks&Rec       -0.107143
ModernFamily    -0.107143
TheOffice        0.476190
Brooklyn99      -0.857143
BigBangTheory    1.142857
30Rock          -0.190476
dtype: float64


In [103]:
#see dataframe for nulls

df2

,Parks&Rec,ModernFamily,TheOffice,Brooklyn99,BigBangTheory,30Rock
Respondent,,,,,,
Phil,3.0,3.0,NaN,1.0,5.0,NaN
Ruth,1.0,NaN,NaN,1.0,NaN,NaN
Jill,2.0,4.0,2.0,NaN,2.0,1.0
Amy,5.0,3.0,4.0,5.0,5.0,3.0
Ian,NaN,1.0,4.0,1.0,NaN,4.0


For each respondent, replace their NaNs with the appropriate global baseline estimate:

In [104]:
#phil
values = {"TheOffice": tv_avg+user_deviation[0]+tv_deviation[2], 
          "30Rock": tv_avg+user_deviation[0]+tv_deviation[5]}
phil=df2.iloc[0].fillna(value=values)

In [105]:
#ruth
values2 = {"ModernFamily": tv_avg+user_deviation[1]+tv_deviation[1], 
           "TheOffice": tv_avg+user_deviation[1]+tv_deviation[2],
           "BigBangTheory": tv_avg+user_deviation[1]+tv_deviation[4],
           "30Rock": tv_avg+user_deviation[1]+tv_deviation[5]
          }
ruth=df2.iloc[1].fillna(value=values2)

In [106]:
#jill
values3 = {"Brooklyn99": tv_avg+user_deviation[2]+tv_deviation[3]}
jill=df2.iloc[2].fillna(value=values3)

#### Amy has no nulls - she's seen all the shows, so nothing to recommend to her

In [107]:
#ian
values4 = {"Parks&Rec": tv_avg+user_deviation[4]+tv_deviation[0],
          "BigBangTheory": tv_avg+user_deviation[4]+tv_deviation[4]}
ian=df2.iloc[4].fillna(value=values4)

#### Recommend a show to a respondent based on the global baseline estimate ratings

In [108]:
import numpy as np

#create list of lists 

lists = np.array([phil, ruth, jill, ian])
lists

array([[3.        , 3.        , 3.47619048, 1.        , 5.        ,
        2.80952381],
       [1.        , 0.89285714, 1.47619048, 1.        , 2.14285714,
        0.80952381],
       [2.        , 4.        , 2.        , 1.34285714, 2.        ,
        1.        ],
       [2.39285714, 1.        , 4.        , 1.        , 3.64285714,
        4.        ]])

In [109]:
#create new df with new ratings (minus Amy because she has none)
new_df = pd.DataFrame(lists, columns= ['Parks&Rec','ModernFamily',
                                        'TheOffice','Brooklyn99',
                                        'BigBangTheory','30Rock'])
#add respondent column
new_df['Respondent']=['Phil','Ruth','Jill','Ian']

#new df called ratings with respondent as the index
ratings=new_df.set_index('Respondent')

ratings

,Parks&Rec,ModernFamily,TheOffice,Brooklyn99,BigBangTheory,30Rock
Respondent,,,,,,
Phil,3.000000,3.000000,3.47619,1.000000,5.000000,2.809524
Ruth,1.000000,0.892857,1.47619,1.000000,2.142857,0.809524
Jill,2.000000,4.000000,2.00000,1.342857,2.000000,1.000000
Ian,2.392857,1.000000,4.00000,1.000000,3.642857,4.000000


I'd recommend the Big Bang Theory to Ruth. She's pretty harsh in her rankings of sitcoms, with most coming in below 1.5. However, her global baseline estimate for the Big Bang Theory gets to a 2.14. If she were to give another sitcom a shot, I'd recommend the Big Bang Theory to her.